In [21]:
# pip install pandas
import pandas as pd

# Read input.csv, skipping the first 5 rows
input_df = pd.read_csv("input.txt", skiprows=4)

# Process data and create the output DataFrame
output_df = pd.DataFrame({
    "Date": input_df["Date"],
    "1": 1,  # Literal value for field 2
    "info 1": "info 1",  # Literal value for field 6
    "Transactions": input_df["Transactions"],
    "Location": input_df["Location"],
    #"Paid out": input_df["Paid out"].str.replace("£", "").apply(lambda x: "-" + x if x else ""),
    #"Paid in": input_df["Paid in"].str.replace("£", "").apply(lambda x: x if x else ""),
    "Paid out": input_df["Paid out"].str.replace("£", ""),
    "Paid in": input_df["Paid in"].str.replace("£", ""),
    "category": "category 1",  # Literal value for field 7
    "Tags": "tag1 tag2"  # Literal value for field 8
})

#output_df["Paid out2"] = output_df["Paid out"].apply(lambda x: "-" + x if x else "")

output_df["Paido"] = output_df["Paid out"].astype(float)
output_df.fillna(0,in

.fillna(0,inplace=True)
output_df["Paidi"] = output_df["Paid in"].astype(float).fillna(0,inplace=True)



# Format the date column
output_df["Date"] = pd.to_datetime(output_df["Date"]).dt.strftime("%y-%m-%d")

# Save output to output.csv
output_df.to_csv("output.csv", sep=";", index=False)

print("got to the end")


got to the end
